In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Project Information
## Data Information
In this data:
* id:ID
* battery_power:Total energy a battery can store in one time measured in mAh
* blue:Has bluetooth or not
* clock_speed:speed at which microprocessor executes instructions
* dual_sim:Has dual sim support or not
* fc:Front Camera mega pixels
* four_g:Has 4G or not
* int_memory:Internal Memory in Gigabytes
* m_dep:Mobile Depth in cm
* mobile_wt:Weight of mobile phone
* n_cores:Number of cores of processor
* pc:Primary Camera mega pixels
* px_height:Pixel Resolution Height
* px_width:Pixel Resolution Width
* ram:Random Access Memory in Megabytes
* sc_h:Screen Height of mobile in cm
* sc_w:Screen Width of mobile in cm
* talk_time:longest time that a single battery charge will last when you are
* three_g:Has 3G or not
* touch_screen:Has touch screen or not
* wifi:Has wifi or not

## Project Aim
In this project, a price range estimation will be made using basic phone features.

# Loading and Examining the Data

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.offline as py
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score,RepeatedStratifiedKFold,RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve, roc_auc_score, recall_score, f1_score,classification_report
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA

In [ ]:
train_df = pd.read_csv("../input/mobile-price-classification/train.csv")
test_df = pd.read_csv("../input/mobile-price-classification/test.csv")
train_df.head()

In [ ]:
train_df.info()

In [ ]:
print("Bluetooth\n",train_df.blue.value_counts())
print("Dual sim\n",train_df.dual_sim.value_counts())
print("4G\n",train_df.four_g.value_counts())
print("Number of Cores\n",train_df.n_cores.value_counts())
print("3G\n",train_df.three_g.value_counts())
print("Touch Screen\n",train_df.touch_screen.value_counts())
print("Wifi\n",train_df.wifi.value_counts())

In [ ]:
train_df.describe().T

In [ ]:
train_df.shape

In [ ]:
test_df.shape

# Data Visualization

## Correlation 
Correlation is a term that is a measure of the strength of a linear relationship between two quantitative variables.Correlation coefficient has a value that must fall between -1.0 and +1.0.
* The closer correlation coefficient is to zero, the weaker the linear relationship.
* Positive correlation coefficient values indicate a positive correlation, where the values of both variables tend to increase together.
* Negative correlation coefficient values indicate a negative correlation, where the values of one variable tend to increase when the values of the other variable decrease.

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(train_df.corr(), annot=True, fmt=".2f");

When the graph above is examined, it is seen that there is a high correlation between RAM and Price Range variables.

## Price range according to the number of processor cores and battery power

In [ ]:
plt.figure(figsize=(15,6));
sns.barplot( x= "n_cores", y = "battery_power" ,hue="price_range", data=train_df)
plt.xticks(rotation=90);

## RAM vs Price Range

In [ ]:
sns.pointplot(y="ram", x="price_range", data=train_df)

In [ ]:
sns.catplot(x="price_range",y="ram",data=train_df)
plt.xticks(rotation=90);

## Internal Memory in Gigabytes vs Price Range

In [ ]:
sns.pointplot(y="int_memory", x="price_range", data=train_df)

## Talk Time vs Price Range

In [ ]:
sns.pointplot(y="talk_time", x="price_range", data=train_df)

## Distribution of Phones Supporting 4G

In [ ]:
total = float(len(train_df))
ax = sns.countplot(train_df["four_g"]);
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}'.format(height/total),
            ha="center"); 

## Distribution of Phones Supporting Wifi

In [ ]:
labels = ["Wifi-supported",'Not supported']
values=train_df['wifi'].value_counts().values
fig1, ax1 = plt.subplots()
ax1.pie(values, labels=labels, autopct='%1.1f%%',shadow=True,startangle=90)
plt.show()

## Distribution of Price Range

In [ ]:
labels = ["0",'1',"2","3"]
values=train_df['price_range'].value_counts().values
fig1, ax1 = plt.subplots()
ax1.pie(values, labels=labels, autopct='%1.1f%%',shadow=True,startangle=90)
plt.show()

## Visualization For Outliers

In [ ]:
def OutliersBox(df, nameOfFeature):
    trace0 = go.Box(y = df[nameOfFeature],
                    name = "All Points",
                    jitter = 0.3,
                    pointpos = -1.8,
                    boxpoints = "all")
    trace1 = go.Box(y = df[nameOfFeature],
                    name = "Only Whiskers",
                    boxpoints = False)
    trace2 = go.Box(y = df[nameOfFeature],
                    name = "Suspected Outliers",
                    boxpoints = "suspectedoutliers",
                    marker = dict(color = 'rgb(8,81,156)',
                                outliercolor = 'rgba(219, 64, 82, 0.6)', line = dict(outliercolor = 'rgba(219, 64, 82, 0.6)',
                                                                                   outlierwidth = 2)),
                    line = dict(color = 'rgb(8,81,156)') )
    trace3 = go.Box(y = df[nameOfFeature],
                    name = "Whiskers and Outliers",
                    boxpoints = "outliers")
    
    data_ = [trace0, trace1, trace2, trace3]
    layout_ = go.Layout(
        title = "{} Outliers".format(nameOfFeature)
    )
    fig = go.Figure(data=data_, layout = layout_)
    py.iplot(fig, filename = "Outliers")

In [ ]:
OutliersBox(train_df,"talk_time")

# Machine Learning Model

## Splitting Data
Above, we first gave all variables except the "price_range" variable to the X variable and gave the variable "price_range" to the y variable. Then we split the data into train and test data. X_train and y_train show the dependent and independent variables to be used to test the model, while X_test and y_test are used to develop the model. Test_size specifies how many of data (10%) will be used for testing. Random_state is used to see the same distinction every time we run the program.

In [ ]:
X = train_df.drop(["price_range"], axis=1)
y = train_df["price_range"]

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.1, random_state=42)

## ML Model Function

In [ ]:
def ml_model(model, parameters):
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=1)
    random_search = RandomizedSearchCV(model, parameters, cv=cv, random_state=1, n_jobs=-1, verbose=2 )
    #grid_search = GridSearchCV(model, parameters, cv=cv, n_jobs=-1, verbose=2)
    pipe = make_pipeline(StandardScaler(), PCA(n_components=0.95),random_search)
    pipe.fit(X_train, y_train)
    
    print('                 Classification report \n')

    print(classification_report(y_test, pipe.predict(X_test)))

## Logistic Regression

In [1]:
log_reg = LogisticRegression()
log_reg_params = {"C" : [1,2,3,0.01,0.001, 2.5, 1.5],
                  "max_iter" : range(100,800,100)}
ml_model(log_reg, log_reg_params)

NameError: name 'LogisticRegression' is not defined

## Decision Tree

In [ ]:
tree = DecisionTreeClassifier()
decTree_params = {"max_depth" : [5,10,15,20,25,30],
                  "min_samples_split" : np.arange(2,50),
                  "min_samples_leaf" : np.arange(1,50)}
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=1)
random_search = RandomizedSearchCV(tree, decTree_params, cv=cv, random_state=1, n_jobs=-1, verbose=2 )
#grid_search = GridSearchCV(model, parameters, cv=cv, n_jobs=-1, verbose=2)
pipe = make_pipeline(random_search)
pipe.fit(X_train, y_train)
    
print('                 Classification report \n')

print(classification_report(y_test, pipe.predict(X_test)))

## KNN

In [ ]:
knn = KNeighborsClassifier()
knn_params = {"n_neighbors" : np.arange(1,50),
              "leaf_size" : np.arange(1,50)}
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=1)
random_search = RandomizedSearchCV(knn, knn_params, cv=cv, random_state=1, n_jobs=-1, verbose=2 )
#grid_search = GridSearchCV(model, parameters, cv=cv, n_jobs=-1, verbose=2)
pipe = make_pipeline(random_search)
pipe.fit(X_train, y_train)
print('                 Classification report \n')

print(classification_report(y_test, pipe.predict(X_test)))

## Random Forest

In [ ]:
forest = RandomForestClassifier()
randomForest_params = {"n_estimators" : [100,500,1000,2000],
                       "min_samples_split" : np.arange(2,30),
                       "min_samples_leaf" : np.arange(1,50),
                       "max_features" : np.arange(1,7)}
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=1)
random_search = RandomizedSearchCV(forest, randomForest_params, cv=cv, random_state=1, n_jobs=-1, verbose=2 )
#grid_search = GridSearchCV(model, parameters, cv=cv, n_jobs=-1, verbose=2)
pipe = make_pipeline(random_search)
pipe.fit(X_train, y_train)
print('                 Classification report \n')

print(classification_report(y_test, pipe.predict(X_test)))